<a href="https://colab.research.google.com/github/minzzii-kim/machine-learing/blob/main/tensorflow_note_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 2-1. 이미지 데이터 분류기 (image)
# 2-2. 정형 데이터 분류기 (structured data)

# [iris 꽃 분류]
# tensorflow datasets 활용한 데이터 전처리 기능 구현 preprocess 함수

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds # 새롭게 추가된 부분

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

train_ds = tfds.load('iris', split='train[:80%]')
valid_ds = tfds.load('iris', split='train[80%:]')

#print(train_ds)

# 시험에서의 전처리 요구조건
# 1. label 값에 대해 원핫인코딩
# 2. features(x), label(y) 로 분할

for data in train_ds.take(5):
  x=data['features']
  y=tf.one_hot(data['label'], 3)
  #print('x: ', x)
  #print('y: ', y)

def preprocess(data):
  x=data['features']
  y=data['label']
  y=tf.one_hot(y, 3)
  return x,y

len(train_ds) #120
len(valid_ds) #30


30

In [11]:
# preprocess 함수를 train_ds 에 적용
# batch_size 적용

# 문제에서 batch_size 주어짐
# 메모리에 120개의 이미지를 한번 올릴수 없기때문에, 1epoch 당 batch 단위로 잘라서 올린다.
# batch 갯수만큼 weight update를 진행하기때문에 성능에 큰 영향을 받는다.

batch_size=10
train_data = train_ds.map(preprocess).batch(batch_size)
valid_data = valid_ds.map(preprocess).batch(batch_size)




In [12]:
# 모델 정의 및 생성
model = Sequential([
  #정형데이터는 이미 1D 이기 때문에 flatten 레이어가 필요없음. 바로 dense 레이어 적용
  Dense(1024, activation='relu'),                    
  Dense(512, activation='relu'),                    
  Dense(256, activation='relu'),                    
  Dense(128, activation='relu'),                    
  Dense(64, activation='relu'),                    
  Dense(32, activation='relu'),                    
  Dense(16, activation='relu'),                    
  Dense(3, activation='softmax'),  # 분류문제의 출력층 => 고정되어있음 => loss 함수 선택                 
])

tmp_path = 'tmp_checkpoint.ckpt'
modelcheckpoint_obj = ModelCheckpoint(
    filepath=tmp_path,
    monitor='val_loss',
    save_weights_only=True,
    save_best_only = True,
    verbose=1
)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])



In [13]:
# 학습시작
model.fit(train_data, validation_data=valid_data, epochs=20, callbacks=[modelcheckpoint_obj] )

model.load_weights(tmp_path)

Epoch 1/20
10/12 [========================>.....] - ETA: 0s - loss: 1.0257 - acc: 0.3700 
Epoch 00001: val_loss improved from inf to 0.88706, saving model to tmp_checkpoint.ckpt
12/12 [==============================] - 2s 36ms/step - loss: 0.9912 - acc: 0.3833 - val_loss: 0.8871 - val_acc: 0.6333
Epoch 2/20
11/12 [==========================>...] - ETA: 0s - loss: 0.7881 - acc: 0.6636
Epoch 00002: val_loss improved from 0.88706 to 0.64021, saving model to tmp_checkpoint.ckpt
12/12 [==============================] - 0s 20ms/step - loss: 0.7688 - acc: 0.6750 - val_loss: 0.6402 - val_acc: 0.7000
Epoch 3/20
10/12 [========================>.....] - ETA: 0s - loss: 0.4842 - acc: 0.7700
Epoch 00003: val_loss improved from 0.64021 to 0.34527, saving model to tmp_checkpoint.ckpt
12/12 [==============================] - 0s 18ms/step - loss: 0.4541 - acc: 0.8000 - val_loss: 0.3453 - val_acc: 0.9000
Epoch 4/20
11/12 [==========================>...] - ETA: 0s - loss: 0.2711 - acc: 0.8818
Epoch 00004

In [15]:
model.evaluate(valid_data)

3/3 [==============================] - 0s 6ms/step - loss: 0.1545 - acc: 0.9333


[0.15446384251117706, 0.9333333373069763]